In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import csv
import json

In [2]:
torch.cuda.empty_cache()  # Clear unus§ed memory

checkpoint = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)  # You may want to use bfloat16 and/or move to GPU here
model.to("cuda")




Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:

# Function to load JSONs from a CSV file
def load_jsons_from_csv(filename):
    data = []  # This list will store all the JSON objects
    errors = []
    with open(filename, 'r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row:  # Ensure the row is not empty
                # Parse JSON from the first column in each row (assuming each row is a JSON string)
                try:
                    json_str = ','.join(part for part in row if part.strip())
                    # print("Json string")
                    # print(json_str)
                    # print(type(json_str))
                    json_data = json.loads(json_str)
                    # print("Json data")
                    # print(json_data)
                    data_list = [json_data]  # Place the dictionary in a list as per your requirement
                    # print("Data list")
                    # print(data_list)
                    data.append(json_data)
                except json.JSONDecodeError:
                    errors.append(row)
                    pass
    print("Number of error rows: " + str(len(errors))) 
    return data

# Example usage
filename = 'Jules_Dialogues_JSON.csv'
loaded_jsons = load_jsons_from_csv(filename)
# print(loaded_jsons)


In [ ]:
print(len(loaded_jsons))
print(len(error))
loaded_jsons

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a very angry comedian and you always get comedically angry when you speak",
    },
    {"role": "user", "content": "How do I make a cup of tea?"},
 ]

In [ ]:
tokenized_chat = tokenizer.apply_chat_template(loaded_jsons, tokenize=True, add_generation_prompt=True, return_tensors="pt")
# print(tokenized_chat)

tokenized_chat = tokenized_chat.to(device)
# # Print the decoded version of the tokenized chat (decoding is done on CPU)
# print(tokenizer.decode(tokenized_chat['input_ids'][0].cpu()))



In [ ]:
# Generate output using the model, ensuring inputs are on the GPU
outputs = model.generate(tokenized_chat, max_new_tokens=1024, pad_token_id=2)
print(tokenizer.decode(outputs[0].cpu(), skip_special_tokens=True))


In [ ]:
# from transformers import pipeline

# pipe = pipeline("text-generation", "HuggingFaceH4/zephyr-7b-beta")
# messages = [
#     {
#         "role": "system",
#         "content": "You are a friendly chatbot who always responds in the style of a pirate",
#     },
#     {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
# ]
# print(pipe(messages, max_new_tokens=128)[0]['generated_text'][-1])  # Print the assistant's response